Specifying tensorflow version

In [ ]:
%tensorflow_version 1.x

Download concra project

In [ ]:
!git clone https://github.com/mrezende/concra.git

Go to StackOverflow-Question-Code-Dataset home

In [ ]:
%cd /content/concra/

Get question ids and codes labeled


In [ ]:
import matplotlib
import pickle
import sys
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import json
from gensim.models import Word2Vec
import random
import numpy as np
import os
from data_processing.code_processing import *
from keras.preprocessing.text import text_to_word_sequence
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import json

iid_labeled = []
with open('data/original/text/python_multi_code_iids.txt','r') as f:
  lines = f.readlines()
  for line in lines:
    iid_labeled.append(line.rstrip())

qid_to_title = json.load(open('data/original/json/python_how_to_do_it_by_classifier_multiple_qid_to_title.json','rb'))

q_code_snippet = json.load(open('data/original/json/python_how_to_do_it_by_classifier_multiple_iid_to_code.json', 'rb'))

all_agreed_iid = json.load(open('data/original/json/all_agreed_iid_to_label.json', 'rb'))





Total of code snippets marked as correct

In [ ]:
iid_marked_as_correct = [key for key, label in all_agreed_iid.items() if label == 1]
len(iid_marked_as_correct)

2169

Calling a function **tokenize_code_corpus**:

Tokenizing a code snippet into a list of tokens.
  Numbers/strings are replaced with NUMBER/STRING.
  Comments are removed. 

  (modified: replacing variable names with VAR)

In [ ]:
qid_code_labeled = dict([(key, q_code_snippet[key]) for key in iid_labeled])

tokenized_code, bool_failed_var, bool_failed_token = tokenize_code_corpus(qid_code_labeled, "python")

Failed line: 
Failed code line: 6  12.34  0.34  b
failed tokenization qid: (36874246, 1)
Failed line: 
Failed code line: (#+[^\\\n]*)
failed tokenization qid: (24829843, 0)
Failed line: 
Failed code line:                       1    2    3
Failed line: 
Failed code line:   4    5    6    7    8    9   10
failed tokenization qid: (19366213, 0)
Failed line: 
Failed code line: abcQdefN
failed tokenization qid: (28617825, 1)
Failed line: 
Failed code line:       ...
Failed line: 
Failed code line: 
Failed line: 
Failed code line:       ...
failed tokenization qid: (28127191, 1)
Failed line: 
Failed code line: response = cli.start(container=container.get('Id'),links=(('EXISTING_CONTAINER', 'LINK_NAME'))
failed tokenization qid: (37144357, 5)
Failed line: 
Failed code line: 
failed tokenization qid: (13696556, 0)
Failed line: 
Failed code line:         print('{:>{w}s}'.format(value, w=len(head)), end=' ')
failed tokenization qid: (34115047, 4)
Failed line: 
Failed code line:     destination.w

Create vector for Tokenizer Class

In [ ]:
import ast
code_samples = [' '.join(tokenized_code[key]) for key in tokenized_code]

# convert iid_labeled keys to tuples. Then we convert to string to get question title
question_samples = [qid_to_title[str(qid)] for qid, code_idx in [ast.literal_eval(key) for key in iid_labeled]]

samples = code_samples + question_samples



In [ ]:
len(samples)

124504

Save samples for tokenizer

In [ ]:
with open('data/samples_for_tokenizer.json', 'w') as write_file:
    json.dump(samples, write_file)

This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary)

In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(samples)

Removing question manually labeled

In [ ]:
print(len(iid_labeled))
print(len(all_agreed_iid))

for key, label in all_agreed_iid.items():
  if label == 1 and key in iid_labeled:
    iid_labeled.remove(key)

print(len(iid_labeled))
print(len(all_agreed_iid))

62252
4884
60083
4884


Collecting questions and code

In [ ]:
sample = {}

for key in iid_labeled:
    qid, code_idx = ast.literal_eval(key)
    qid = str(qid)

    if qid in sample:
      sample[qid]['code_snippets'].append(' '.join(tokenized_code[key]))
    else:
      sample[qid] = {}
      sample[qid]['question'] = qid_to_title[qid]
      sample[qid]['code_snippets'] = [' '.join(tokenized_code[key])]

In [ ]:
sum([len(sample[qid]['code_snippets']) for qid in sample])

60083

Checking if text_to_sequences is working

In [ ]:
text = sample['13653334']['question']
print('Question: %s' % text)
print("Vector tokenized: %s" % tokenizer.texts_to_sequences([text])[0])
word_index = tokenizer.word_index
inv_word_index = {v: k for k, v in word_index.items()}
reverse_token = [inv_word_index[k] for k in tokenizer.texts_to_sequences([text])[0]]
print('Tokenized reversed: %s' % reverse_token)

# it works for the first question as they dont have accent. 
# Tokenizer class remove accents and special characters. 
assert text.lower() == ' '.join(reverse_token)


Question: Python string formatted in double qoutes for JS
Vector tokenized: [13, 4, 1728, 5, 696, 34223, 6, 3996]
Tokenized reversed: ['python', 'string', 'formatted', 'in', 'double', 'qoutes', 'for', 'js']


Collecting answers

In [ ]:
answers = []
for key in iid_labeled:
    answers.append(' '.join(tokenized_code[key]))

Tokenize answers

In [ ]:
answers_sequence = tokenizer.texts_to_sequences(answers)

Save tokenized answers

In [ ]:
with open('data/answers.json', 'w') as write_file:
    json.dump(answers_sequence, write_file)

Questions and code tokenized to a sequence of integers

In [ ]:
sample_sequence = []

for key in sample:
  question = sample[key]['question']
  question_sequence = tokenizer.texts_to_sequences([question])[0]
  
  code_snippets = sample[key]['code_snippets']
  code_snippets_sequence = tokenizer.texts_to_sequences(code_snippets)
  

  
  input_sequence = {'question': question_sequence, 'good_answers': code_snippets_sequence}
  
  sample_sequence.append(input_sequence)


Creating training dataset.

Its dataset contains question and good answers labeled. According to https://github.com/mrezende/StackOverflow-Question-Code-Dataset

In [ ]:
qid, code_idx = ast.literal_eval(iid_labeled[0])
question_url = f'https://stackoverflow.com/questions/{qid}'
print(question_url)
print(sample_sequence[0]['question'])
print(len(sample_sequence[0]['good_answers']))
print(len(sample_sequence))
print([inv_word_index[key] for key in sample_sequence[0]['question']])
print([[inv_word_index[key] for key in code_snippet] for code_snippet in sample_sequence[0]['good_answers']])

https://stackoverflow.com/questions/17335003
[13, 979, 14, 153, 26, 727, 18, 25, 81, 45, 45, 45]
2
39136
['python', 'trying', 'to', 'create', 'the', 'equivalent', 'of', 'and', 'or', 'value', 'value', 'value']
[['var', 'set', 'string', 'string', 'newline', 'var', 'set', 'string', 'set', 'string', 'string', 'string', 'set', 'string', 'string', 'string', 'string', 'string', 'string', 'string', 'string', 'newline', 'def', 'test', 'teststring', 'newline', 'if', 'not', 'teststring', 'endswith', 'string', 'newline', 'return', 'false', 'newline', 'var', 'set', 'teststring', 'rstrip', 'string', 'lower', 'split', 'newline', 'return', 'var', 'issuperset', 'var', 'or', 'all', 'not', 'var', 'isdisjoint', 'var', 'for', 'var', 'in', 'var', 'newline', 'var', 'string', 'newline', 'var', 'string', 'newline', 'var', 'string', 'newline', 'var', 'string', 'newline', 'var', 'string', 'newline', 'print', 'var', 'newline', 'if', 'test', 'var', 'newline', 'print', 'var', 'newline', 'print', 'var', 'newline', '

Save training dataset

In [ ]:
with open('data/training.json', 'w') as write_file:
    json.dump(sample_sequence, write_file)

Collecting question and code labeled as 1 ( 'correct' )

In [ ]:
sample_for_evaluation = {}

for key, label in all_agreed_iid.items():
    qid, code_idx = ast.literal_eval(key)
    qid = str(qid)
    if label == 1:
      if qid in sample_for_evaluation:
        sample_for_evaluation[qid]['code_snippets'].append(' '.join(tokenized_code[key]))
      else:
        sample_for_evaluation[qid] = {}
        sample_for_evaluation[qid]['question'] = qid_to_title[qid]
        sample_for_evaluation[qid]['code_snippets'] = [' '.join(tokenized_code[key])]

In [ ]:
sum([len(sample_for_evaluation[qid]['code_snippets']) for qid in sample_for_evaluation])


2169

Questions and code labeled tokenized

In [ ]:
sample_for_evaluation_sequence = []

for key in sample_for_evaluation:
  question = sample_for_evaluation[key]['question']
  question_sequence = tokenizer.texts_to_sequences([question])[0]
  
  code_snippets = sample_for_evaluation[key]['code_snippets']
  code_snippets_sequence = tokenizer.texts_to_sequences(code_snippets)
  
  

  
  input_sequence = {'question': question_sequence, 'good_answers': code_snippets_sequence}
  
  sample_for_evaluation_sequence.append(input_sequence)

Create DEV and EVAL set

In [ ]:
sum([len(input_sequence['good_answers']) for input_sequence in sample_for_evaluation_sequence])

2169

Split data in DEV and EVAL

In [ ]:
from sklearn.model_selection import train_test_split
X_dev, X_eval = train_test_split(
    sample_for_evaluation_sequence, test_size=0.50, random_state=42)

In [ ]:
len(X_eval)

721

Save DEV dataset

In [ ]:
with open('data/dev.json', 'w') as write_file:
    json.dump(X_dev, write_file) 

Save EVAL dataset

In [ ]:
with open('data/eval.json', 'w') as write_file:
    json.dump(X_eval, write_file) 

Compress files

In [ ]:
%cd data
!tar -czvf samples_for_tokenizer.tar.gz samples_for_tokenizer.json
!tar -czvf answers.tar.gz answers.json
!tar -czvf training.tar.gz training.json 
!tar -czvf eval.tar.gz eval.json
!tar -czvf dev.tar.gz dev.json

/content/yacra/data
samples_for_tokenizer.json
answers.json
training.json
eval.json
dev.json


compress in a single file

In [ ]:
!tar -czvf data.tar.gz *.tar.gz* 

answers.tar.gz
dev.tar.gz
eval.tar.gz
samples_for_tokenizer.tar.gz
training.tar.gz
